In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

In [2]:
sk_names_lst = []
g_names_lst = []
pos_lst = []
goals_lst = []
assists_lst = []
points_lst = []
ops_lst = []
tps_lst = []
plus_minus_lst = []
gps_lst = []
gwins_lst = []
gaa_lst = []
sv_lst = []

In [3]:
# 2025 skater data
response = requests.get('https://www.hockey-reference.com/leagues/NHL_2025_skaters-misc.html')
print(response.status_code)

200


In [4]:
soup = bs(response.content, 'html.parser')

In [5]:
sk_cols = [sk_names_lst, pos_lst, goals_lst, assists_lst, points_lst, ops_lst, tps_lst, plus_minus_lst]

tags = ['th+ .left', '.left+ .left', '.right:nth-child(7)', '.right:nth-child(8)', '.right:nth-child(9)',
        '.right:nth-child(16)', '.right:nth-child(18)', '.right:nth-child(15)']

In [6]:
for col, tag in zip(sk_cols, tags):
    temp = soup.select(tag)
    for t in temp:
        col.append(t.string)

In [7]:
sk_data = {}

keys = ['player names', 'position', 'goals', 'assists', 'points', 'ops', 'tps', 'plus-minus']

for i in range(len(keys)):
    sk_data[keys[i]] = sk_cols[i]

sk_df = pd.DataFrame(sk_data)

In [8]:
sk_df['points'] = sk_df['points'].apply(lambda x: int(x) if isinstance(x, str) else 0)

In [9]:
sk_df = sk_df.nlargest(15, 'points', keep='all')
sk_df.reset_index(inplace=True, drop=True)

In [10]:
sk_df[['goals', 'assists', 'plus-minus']] = sk_df[['goals', 'assists', 'plus-minus']].astype(int)
sk_df[['ops', 'tps']] = sk_df[['ops', 'tps']].astype(float)

In [11]:
sk_df['position'] = sk_df['position'].replace(['RW', 'LW'], ['W', 'W'])

In [12]:
sk_df = pd.get_dummies(sk_df, columns=['position'], dtype=int)
sk_df.head(10)

,player names,goals,assists,points,ops,tps,plus-minus,position_C,position_D,position_W
0,Nathan MacKinnon,28,99,127,3.5,6.0,4,1,0,0
1,Martin Nečas,42,85,127,4.1,5.9,5,1,0,0
2,Kirill Kaprizov,54,73,127,4.4,6.7,21,0,0,1
3,Nikita Kucherov,42,79,121,3.6,5.2,6,0,0,1
4,Mikko Rantanen,50,66,116,4.0,6.3,3,0,0,1
5,Jack Eichel,27,88,115,3.1,5.2,12,1,0,0
6,Leon Draisaitl,60,53,113,4.1,6.4,15,1,0,0
7,Connor McDavid,42,71,113,3.6,5.5,8,1,0,0
8,Sam Reinhart,55,57,112,4.0,6.3,9,1,0,0
9,Mitch Marner,30,82,112,3.2,5.1,5,0,0,1


In [13]:
sk_df[['ops', 'tps']] = sk_df[['ops', 'tps']] * 2.5
sk_df['plus-minus'] = sk_df['plus-minus'] * 2

In [14]:
order = ['player names', 'goals', 'assists', 'ops', 'plus-minus', 'points', 'tps', 'position_C', 'position_D', 'position_W']
sk_df = sk_df[order]
sk_df.head(10)

,player names,goals,assists,ops,plus-minus,points,tps,position_C,position_D,position_W
0,Nathan MacKinnon,28,99,8.75,8,127,15.00,1,0,0
1,Martin Nečas,42,85,10.25,10,127,14.75,1,0,0
2,Kirill Kaprizov,54,73,11.00,42,127,16.75,0,0,1
3,Nikita Kucherov,42,79,9.00,12,121,13.00,0,0,1
4,Mikko Rantanen,50,66,10.00,6,116,15.75,0,0,1
5,Jack Eichel,27,88,7.75,24,115,13.00,1,0,0
6,Leon Draisaitl,60,53,10.25,30,113,16.00,1,0,0
7,Connor McDavid,42,71,9.00,16,113,13.75,1,0,0
8,Sam Reinhart,55,57,10.00,18,112,15.75,1,0,0
9,Mitch Marner,30,82,8.00,10,112,12.75,0,0,1


In [15]:
sk_df.to_csv('2025_sk_forecast.csv')

In [16]:
# 2025 goalie data
response = requests.get('https://www.hockey-reference.com/leagues/NHL_2025_goalies.html')
response.status_code

200

In [17]:
soup = bs(response.content, 'html.parser')

In [18]:
g_cols = [g_names_lst, gps_lst, gwins_lst, gaa_lst, sv_lst]

tags = ['th+ .left', '.right:nth-child(24)', '.right:nth-child(8)', '.non_qual+ .non_qual', '.non_qual:nth-child(14)']

In [19]:
for col, tag in zip(g_cols, tags):
    temp = soup.select(tag)
    for t in temp:
        col.append(t.string)

In [20]:
g_data = {}

keys = ['player names', 'gps', 'wins', 'gaa', 'sv%']

for i in range(len(keys)):
    g_data[keys[i]] = g_cols[i]

g_df = pd.DataFrame(g_data)
g_df

,player names,gps,wins,gaa,sv%
0,Juuse Saros,4.7,6,2.72,.908
1,Connor Hellebuyck,5.7,18,2.09,.927
2,Sam Montembeault,3.9,9,2.83,.906
3,Andrei Vasilevskiy,4.0,12,2.38,.910
4,Jordan Binnington,3.3,8,2.93,.897
...,...,...,...,...,...
84,Yaniv Perets,0.0,0,7.32,.857
85,Erik Portillo,0.4,1,1.02,.966
86,Akira Schmid,0.2,0,.00,1.000
87,Mads Søgaard,-0.1,1,6.96,.765


In [21]:
g_df[['gps', 'gaa', 'sv%']] = g_df[['gps', 'gaa', 'sv%']].astype(float)

In [22]:
g_df['wins'] = g_df['wins'].apply(lambda x: int(x) if isinstance(x, str) else 0)

In [23]:
g_df = g_df.nlargest(5, 'gps', keep='all')
g_df.reset_index(inplace=True, drop=True)

In [24]:
g_df[['gps', 'wins']] = g_df[['gps', 'wins']] * 3

In [25]:
g_df.to_csv('2025_g_forecast.csv')